In [ ]:
import pandas as  pd
import requests

# 페이지 번호를 제외한 주소 저장
new_url = 'https://finance.naver.com/item/sise_day.naver?code=086520&page='

# 헤더 정보 설정하기
my_headers = {'user-agent': 'Mozilla/5.0'}
# 데이터를 축적할 데이터프레임 생성
all_tables_year = pd.DataFrame()

# 페이지 번호 증가용 반복문
for page_number in range(1, 100):

# 페이지 번호 추가한 주소 완성
    full_url = new_url + str(page_number)

# 주소 확인하기
    print(f'{page_number} 번째 페이지 읽어오기({full_url})')

# HTTP 요청 전송 후 응답 받아오기
    page = requests.get(full_url, headers=my_headers)

# 테이블 추출
    table = pd.read_html(page.text)[0]

# 수행할 내용 확인
    # print(f'전체 {len(all_tables_year.index)} 줄에 {len(table.index)} 줄 추가')

# 데이터 축적용 데이터프레임에 테이블 추가
    all_tables_year = pd.concat([all_tables_year, table])
    print(all_tables_year)

In [ ]:
from prophet import Prophet

In [ ]:
all_tables_year.dropna(inplace=True)

In [ ]:
# 열 이름 영문으로 변경
all_tables_year.rename(columns={'날짜':'ds'}
                       , inplace=True)
# 중위값 열 추가
all_tables_year['y'] = all_tables_year['고가'] - ((all_tables_year['고가'] - all_tables_year['저가']) / 2)

all_tables_year.head()

In [ ]:
df = pd.DataFrame()
#필요한 열만 추출
df = all_tables_year[['ds', 'y']]
df.info()

In [ ]:
# 프로핏 모델 객체 만들기
m = Prophet()
# 데이터프레임 학습
m.fit(df)

In [ ]:
# 예측 데이터프레임 준비하기
future = m.make_future_dataframe(periods=1000)

# 끝부분 확인하기
future.tail()

In [ ]:
# 예측하기
forecast = m.predict(future)
# 예측결과 끝부분 확인하기
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

In [ ]:
fig1 = m.plot(forecast)

In [ ]:
# 2차 예측을 위한 프로핏 객체 만들기
m2 = Prophet()

# 2021년 1월 1일 이후 데이터만 준비
df2 = df[df['ds'] > '2021-01-01']

# 데이터 끝 부분 확인하기
df2.tail()

In [ ]:
# 데이터 학습하기
m2.fit(df2)

# 예측 데이터프레임 만들기
future2 = m2.make_future_dataframe(periods=1000)

# 예측하기
forecast2 = m2.predict(future2)

# 예측 결과 끝부분 확인하기
forecast2[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

# 시각화하기
fig3 = m2.plot(forecast2)
# 2차 예측 컴포넌트 확인하기
fig4 = m2.plot_components(forecast2)

In [ ]:
# plot 모듈 탑재
from prophet.plot import plot_plotly, ipywidgets
m2 = Prophet()
# 대화형 예측 그래프 그리기
plot_plotly(m2, forecast2)